In [2]:
import torch
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from torchvision import transforms
import torchvision
import numpy as np
from torch.utils.data import DataLoader

In [3]:
transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5),
                                                     (0.5, 0.5, 0.5))])
dataset = ImageFolder('/kaggle/input/iais22-birds/birds/birds', transform=transform)
classes = dataset.classes
print(f"Hay {len(classes)} neuronas en nuestra red")
data_loader = DataLoader(dataset, batch_size=20, shuffle=True)


In [4]:
def imshow(img):
    img = img / 2 + 0.5 
    plt.imshow(np.transpose(img, (1, 2, 0)))
dataiter = iter(data_loader)
images, labels = dataiter.next()
fig = plt.figure(figsize=(30, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20//2, idx+1, xticks=[], yticks=[])
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])

In [5]:
labels = {}

for label in classes:
    labels[label] = 0

data_loader = DataLoader(dataset, batch_size=1, shuffle=True)

for data in data_loader:
    img, label = data
    labels[classes[label.item()]] += 1
print(labels)

In [6]:
from torch.utils.data import random_split
train_set, test_set = random_split(dataset, (int(len(dataset) * 0.7) + 1, int(len(dataset) * 0.3)))
train_set, valid_set = random_split(train_set, (int(len(train_set) * 0.7) + 1, int(len(train_set) * 0.3)))

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(46656, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 400)

    def forward(self, x):
        
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [16]:
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

cnn = Net().to(device)

criterion = nn.CrossEntropyLoss()
# params = cnn.resnet.fc.parameters()
params = cnn.parameters()
optimizer = optim.Adam(params, lr=0.003)

In [17]:
train_loader = DataLoader(train_set, batch_size=128)
test_loader = DataLoader(test_set, batch_size=1)

In [18]:
print("modelo ", cnn)
print("train loader ", train_loader)
print("criterion ",criterion)
print("optimizer ",optimizer)
print("device ", device)

In [19]:
def train_model(model, train_loader, criterion, optimizer, device):
    total_step = len(train_loader)
    num_epochs = 10
    
    for epoch in range(num_epochs):
        train_loss = 0.0
        

        model.train()
    for i, (img, target) in enumerate(train_loader):
        img = img.to(device)
        target = target.to(device)

        optimizer.zero_grad()

        output = model(img)


        loss = criterion(output, target)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * img.size(0)

    model.eval()
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target)
        train_loss = train_loss / len(train_loader.sampler)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
          epoch, train_loss))

In [20]:
print("PARAMETROS QUE ENTRAN A train_model MENOS train_loader:")

print("model:\n ", cnn)
print("criterion:\n ",criterion)
print("optimizer:\n ",optimizer)
print("device:\n ", device)

In [21]:
train_model(cnn, train_loader, criterion, optimizer, device)

In [49]:
def global_accuracy(model, test_loader):
    correct = 0
    total = 0
    model.to("cpu")
    dataiter = iter(test_loader)
    with torch.no_grad():
        for data in dataiter:
            img, label = data
            m = nn.Softmax(dim=1)
            output = m(model(img))
            _, predicted = torch.max(output.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()

    print(f"Accuracy: {100 * correct / total}")

def accuracy_per_class(model, test_loader, classes, device):
    class_correct = list(0. for i in range(5))
    class_total = list(0. for i in range(5))
    cnn.to(device)
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = cnn(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            if(c.item()):
                class_correct[labels.item()] += 1
                class_total[labels.item()] += 1


    for i in range(5):

        print(f"{classes[i]} | Correct: {class_correct[i]} | Total: {class_total[i]}" +
          f" | Accuracy: {class_correct[i] / class_total[i]}")

    

In [50]:
global_accuracy(cnn, test_loader)
'accuracy_per_class(cnn, test_loader, classes, device)'